In [1]:
"""
Evaluation & Diagnostic Plots for Tidal Volume Prediction
Saves plots to disk and displays metrics.

Requirements:
pip install pandas numpy matplotlib scikit-learn joblib
"""

import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import joblib
import math

# --------- USER CONFIG ----------
# Option A: If you have a CSV that already contains true and predicted values
# Set csv_path and column names appropriately
csv_path = "predictions.csv"   # e.g. contains columns: 'y_true', 'y_pred'
y_true_col = "y_true"
y_pred_col = "y_pred"

# Option B: If you have a saved model and test data (uncomment and edit)
model_path = "rf_model.joblib"     # serialized scikit-learn model
X_test_path = "X_test.csv"         # features used for prediction
y_test_path = "y_test.csv"         # true target values (single column or dataframe)

# Output folder for images
out_dir = "evaluation_plots"
os.makedirs(out_dir, exist_ok=True)
# ---------------------------------

def load_from_csv(path, y_true_col, y_pred_col):
    df = pd.read_csv(path)
    if y_true_col not in df.columns or y_pred_col not in df.columns:
        raise ValueError(f"CSV must contain columns '{y_true_col}' and '{y_pred_col}'. Columns found: {list(df.columns)}")
    return df[y_true_col].to_numpy(), df[y_pred_col].to_numpy(), df

def load_from_model(model_path, X_test_path, y_test_path):
    model = joblib.load(model_path)
    X_test = pd.read_csv(X_test_path)
    # if y_test in same file as X_test, change accordingly
    y_test_df = pd.read_csv(y_test_path)
    # assume y_test is single column
    if y_test_df.shape[1] == 1:
        y_test = y_test_df.iloc[:,0].to_numpy()
    else:
        # try column named 'y' or 'y_true' etc.
        candidates = ['y', 'y_true', 'Actual', 'Actual_VT', 'true_vt']
        found = None
        for c in candidates:
            if c in y_test_df.columns:
                found = c
                break
        if found:
            y_test = y_test_df[found].to_numpy()
        else:
            # take first column
            y_test = y_test_df.iloc[:,0].to_numpy()

    # Predict
    y_pred = model.predict(X_test)
    return y_test, y_pred, X_test, model

def compute_metrics(y_true, y_pred):
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = math.sqrt(mean_squared_error(y_true, y_pred))
    return {"R2": r2, "MAE": mae, "RMSE": rmse}

def plot_diagnostics(y_true, y_pred, df_meta=None, out_dir="evaluation_plots", prefix="eval"):
    metrics = compute_metrics(y_true, y_pred)
    # Pred vs Actual scatter
    plt.figure(figsize=(8,6))
    plt.scatter(y_true, y_pred, s=30, alpha=0.6)
    minv = min(min(y_true), min(y_pred))
    maxv = max(max(y_true), max(y_pred))
    padding = (maxv - minv) * 0.05
    plt.plot([minv-padding, maxv+padding], [minv-padding, maxv+padding], 'k--', linewidth=1)  # y=x line
    plt.xlabel("Actual Tidal Volume (ml)")
    plt.ylabel("Predicted Tidal Volume (ml)")
    plt.title("Predicted vs Actual - Tidal Volume")
    # show metrics text box
    textbox = '\n'.join([f"{k}: {v:.3f}" if k!='MAE' and k!='RMSE' else f"{k}: {v:.2f} ml" 
                         for k,v in metrics.items()])
    plt.gca().text(0.02, 0.98, textbox, transform=plt.gca().transAxes, fontsize=10,
                   verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    plt.grid(alpha=0.3)
    fname1 = os.path.join(out_dir, f"{prefix}_pred_vs_actual.png")
    plt.tight_layout()
    plt.savefig(fname1, dpi=150)
    plt.close()

    # Residuals histogram
    residuals = y_true - y_pred
    plt.figure(figsize=(8,5))
    plt.hist(residuals, bins=30, alpha=0.8)
    plt.xlabel("Residuals (Actual - Predicted) [ml]")
    plt.title("Residuals Distribution")
    plt.grid(alpha=0.3)
    fname2 = os.path.join(out_dir, f"{prefix}_residuals_hist.png")
    plt.tight_layout()
    plt.savefig(fname2, dpi=150)
    plt.close()

    # Residuals vs Predicted
    plt.figure(figsize=(8,5))
    plt.scatter(y_pred, residuals, s=25, alpha=0.6)
    plt.axhline(0, color='k', linestyle='--')
    plt.xlabel("Predicted Tidal Volume (ml)")
    plt.ylabel("Residuals (Actual - Predicted) [ml]")
    plt.title("Residuals vs Predicted")
    plt.grid(alpha=0.3)
    fname3 = os.path.join(out_dir, f"{prefix}_residuals_vs_pred.png")
    plt.tight_layout()
    plt.savefig(fname3, dpi=150)
    plt.close()

    # Error vs Feature (optional) - if df_meta present and has 'BMI' or 'Height', plot relationship
    if df_meta is not None:
        # try plotting error vs top few features if present
        for col in ["BMI", "Height", "Weight", "PaO2", "SpO2"]:
            if col in df_meta.columns:
                plt.figure(figsize=(7,4))
                plt.scatter(df_meta[col], residuals, s=25, alpha=0.6)
                plt.axhline(0, color='k', linestyle='--')
                plt.xlabel(col)
                plt.ylabel("Residual (ml)")
                plt.title(f"Residuals vs {col}")
                plt.grid(alpha=0.3)
                fname = os.path.join(out_dir, f"{prefix}_residuals_vs_{col}.png")
                plt.tight_layout()
                plt.savefig(fname, dpi=150)
                plt.close()

    print("Saved plots to:", os.path.abspath(out_dir))
    return metrics, [fname1, fname2, fname3]

def main():
    # Try Option A first: CSV with y_true & y_pred
    if os.path.exists(csv_path):
        try:
            y_true, y_pred, df = load_from_csv(csv_path, y_true_col, y_pred_col)
            # If there are other columns in df, we can use for meta plots
            metrics, files = plot_diagnostics(y_true, y_pred, df_meta=df, out_dir=out_dir, prefix="predcsv")
            print("Metrics (from CSV):", metrics)
            return
        except Exception as e:
            print("CSV load failed:", e)
    # If CSV not available or failed, try Option B: load model & X_test
    if os.path.exists(model_path) and os.path.exists(X_test_path) and os.path.exists(y_test_path):
        try:
            y_test, y_pred, X_test, model = load_from_model(model_path, X_test_path, y_test_path)
            metrics, files = plot_diagnostics(y_test, y_pred, df_meta=X_test, out_dir=out_dir, prefix="from_model")
            print("Metrics (from model):", metrics)
            return
        except Exception as e:
            print("Model-based evaluation failed:", e)

    # If neither option worked, show example using random data (demo)
    print("No CSV or model/test data found. Running demo with synthetic example.")
    np.random.seed(0)
    y_true = np.random.normal(loc=450, scale=50, size=200)  # e.g., actual vt ml
    noise = np.random.normal(loc=0, scale=20, size=200)
    y_pred = y_true + noise
    metrics, files = plot_diagnostics(y_true, y_pred, df_meta=None, out_dir=out_dir, prefix="demo")
    print("Demo metrics:", metrics)

if __name__ == "__main__":
    main()


No CSV or model/test data found. Running demo with synthetic example.
Saved plots to: e:\venti\evaluation_plots
Demo metrics: {'R2': 0.8610378944213931, 'MAE': 15.116888810886513, 'RMSE': 19.038105620423053}


In [2]:
"""
Evaluation & Diagnostic Plots for Tidal Volume Prediction
Saves plots to disk and displays metrics.

Requirements:
pip install pandas numpy matplotlib scikit-learn joblib
"""

import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import joblib
import math

# --------- USER CONFIG ----------
# Option A: If you have a CSV that already contains true and predicted values
# Set csv_path and column names appropriately
csv_path = "predictions.csv"   # e.g. contains columns: 'y_true', 'y_pred'
y_true_col = "y_true"
y_pred_col = "y_pred"

# Option B: If you have a saved model and test data (uncomment and edit)
model_path = "rf_model.joblib"     # serialized scikit-learn model
X_test_path = "X_test.csv"         # features used for prediction
y_test_path = "y_test.csv"         # true target values (single column or dataframe)

# Output folder for images
out_dir = "evaluation_plots"
os.makedirs(out_dir, exist_ok=True)
# ---------------------------------

def load_from_csv(path, y_true_col, y_pred_col):
    df = pd.read_csv(path)
    if y_true_col not in df.columns or y_pred_col not in df.columns:
        raise ValueError(f"CSV must contain columns '{y_true_col}' and '{y_pred_col}'. Columns found: {list(df.columns)}")
    return df[y_true_col].to_numpy(), df[y_pred_col].to_numpy(), df

def load_from_model(model_path, X_test_path, y_test_path):
    model = joblib.load(model_path)
    X_test = pd.read_csv(X_test_path)
    # if y_test in same file as X_test, change accordingly
    y_test_df = pd.read_csv(y_test_path)
    # assume y_test is single column
    if y_test_df.shape[1] == 1:
        y_test = y_test_df.iloc[:,0].to_numpy()
    else:
        # try column named 'y' or 'y_true' etc.
        candidates = ['y', 'y_true', 'Actual', 'Actual_VT', 'true_vt']
        found = None
        for c in candidates:
            if c in y_test_df.columns:
                found = c
                break
        if found:
            y_test = y_test_df[found].to_numpy()
        else:
            # take first column
            y_test = y_test_df.iloc[:,0].to_numpy()

    # Predict
    y_pred = model.predict(X_test)
    return y_test, y_pred, X_test, model

def compute_metrics(y_true, y_pred):
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = math.sqrt(mean_squared_error(y_true, y_pred))
    return {"R2": r2, "MAE": mae, "RMSE": rmse}

def plot_diagnostics(y_true, y_pred, df_meta=None, out_dir="evaluation_plots", prefix="eval"):
    metrics = compute_metrics(y_true, y_pred)
    # Pred vs Actual scatter
    plt.figure(figsize=(8,6))
    plt.scatter(y_true, y_pred, s=30, alpha=0.6)
    minv = min(min(y_true), min(y_pred))
    maxv = max(max(y_true), max(y_pred))
    padding = (maxv - minv) * 0.05
    plt.plot([minv-padding, maxv+padding], [minv-padding, maxv+padding], 'k--', linewidth=1)  # y=x line
    plt.xlabel("Actual Tidal Volume (ml)")
    plt.ylabel("Predicted Tidal Volume (ml)")
    plt.title("Predicted vs Actual - Tidal Volume")
    # show metrics text box
    textbox = '\n'.join([f"{k}: {v:.3f}" if k!='MAE' and k!='RMSE' else f"{k}: {v:.2f} ml" 
                         for k,v in metrics.items()])
    plt.gca().text(0.02, 0.98, textbox, transform=plt.gca().transAxes, fontsize=10,
                   verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    plt.grid(alpha=0.3)
    fname1 = os.path.join(out_dir, f"{prefix}_pred_vs_actual.png")
    plt.tight_layout()
    plt.savefig(fname1, dpi=150)
    plt.close()

    # Residuals histogram
    residuals = y_true - y_pred
    plt.figure(figsize=(8,5))
    plt.hist(residuals, bins=30, alpha=0.8)
    plt.xlabel("Residuals (Actual - Predicted) [ml]")
    plt.title("Residuals Distribution")
    plt.grid(alpha=0.3)
    fname2 = os.path.join(out_dir, f"{prefix}_residuals_hist.png")
    plt.tight_layout()
    plt.savefig(fname2, dpi=150)
    plt.close()

    # Residuals vs Predicted
    plt.figure(figsize=(8,5))
    plt.scatter(y_pred, residuals, s=25, alpha=0.6)
    plt.axhline(0, color='k', linestyle='--')
    plt.xlabel("Predicted Tidal Volume (ml)")
    plt.ylabel("Residuals (Actual - Predicted) [ml]")
    plt.title("Residuals vs Predicted")
    plt.grid(alpha=0.3)
    fname3 = os.path.join(out_dir, f"{prefix}_residuals_vs_pred.png")
    plt.tight_layout()
    plt.savefig(fname3, dpi=150)
    plt.close()

    # Error vs Feature (optional) - if df_meta present and has 'BMI' or 'Height', plot relationship
    if df_meta is not None:
        # try plotting error vs top few features if present
        for col in ["BMI", "Height", "Weight", "PaO2", "SpO2"]:
            if col in df_meta.columns:
                plt.figure(figsize=(7,4))
                plt.scatter(df_meta[col], residuals, s=25, alpha=0.6)
                plt.axhline(0, color='k', linestyle='--')
                plt.xlabel(col)
                plt.ylabel("Residual (ml)")
                plt.title(f"Residuals vs {col}")
                plt.grid(alpha=0.3)
                fname = os.path.join(out_dir, f"{prefix}_residuals_vs_{col}.png")
                plt.tight_layout()
                plt.savefig(fname, dpi=150)
                plt.close()

    print("Saved plots to:", os.path.abspath(out_dir))
    return metrics, [fname1, fname2, fname3]

def main():
    # Try Option A first: CSV with y_true & y_pred
    if os.path.exists(csv_path):
        try:
            y_true, y_pred, df = load_from_csv(csv_path, y_true_col, y_pred_col)
            # If there are other columns in df, we can use for meta plots
            metrics, files = plot_diagnostics(y_true, y_pred, df_meta=df, out_dir=out_dir, prefix="predcsv")
            print("Metrics (from CSV):", metrics)
            return
        except Exception as e:
            print("CSV load failed:", e)
    # If CSV not available or failed, try Option B: load model & X_test
    if os.path.exists(model_path) and os.path.exists(X_test_path) and os.path.exists(y_test_path):
        try:
            y_test, y_pred, X_test, model = load_from_model(model_path, X_test_path, y_test_path)
            metrics, files = plot_diagnostics(y_test, y_pred, df_meta=X_test, out_dir=out_dir, prefix="from_model")
            print("Metrics (from model):", metrics)
            return
        except Exception as e:
            print("Model-based evaluation failed:", e)

    # If neither option worked, show example using random data (demo)
    print("No CSV or model/test data found. Running demo with synthetic example.")
    np.random.seed(0)
    y_true = np.random.normal(loc=450, scale=50, size=200)  # e.g., actual vt ml
    noise = np.random.normal(loc=0, scale=20, size=200)
    y_pred = y_true + noise
    metrics, files = plot_diagnostics(y_true, y_pred, df_meta=None, out_dir=out_dir, prefix="demo")
    print("Demo metrics:", metrics)

if __name__ == "__main__":
    main()


No CSV or model/test data found. Running demo with synthetic example.
Saved plots to: e:\venti\evaluation_plots
Demo metrics: {'R2': 0.8610378944213931, 'MAE': 15.116888810886513, 'RMSE': 19.038105620423053}


In [3]:
"""
evaluate_and_plot.py
Train RandomForest, compute MAE/RMSE/R2, save useful plots:
 - performance_metrics.png (MAE & R2 bar)
 - rmse_text.png (RMSE displayed)
 - predicted_vs_actual.png
 - residuals_plot.png
 - learning_curve.png
 - feature_importance.png
 - shap_summary.png (if SHAP available)
Also saves model as tv_predictor_model.pkl
"""

import os
import math
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score, learning_curve
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# ---------- User settings ----------
CSV_PATH = "icu_simulated_data.csv"
TARGET_COL = "TV_recommendation"    # change if target column name differs
MODEL_OUT = "tv_predictor_model.pkl"
OUTDIR = "plots"
RANDOM_STATE = 42
# -----------------------------------

os.makedirs(OUTDIR, exist_ok=True)

# 1) Load data
df = pd.read_csv(CSV_PATH)
if TARGET_COL not in df.columns:
    raise ValueError(f"Target column '{TARGET_COL}' not found in {CSV_PATH}")

X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL]

# Simple preprocessing: If there are non-numeric columns, try one-hot encode them
X = pd.get_dummies(X, drop_first=True)

# 2) Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE
)

# 3) Train model
model = RandomForestRegressor(n_estimators=200, max_depth=10, random_state=RANDOM_STATE)
model.fit(X_train, y_train)

# Save model
joblib.dump(model, MODEL_OUT)
print(f"Model saved to {MODEL_OUT}")

# 4) Predictions & metrics
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = math.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("MAE: {:.4f} ml".format(mae))
print("RMSE: {:.4f} ml".format(rmse))
print("R2: {:.4f}".format(r2))

# 5) Plot: MAE & R2 bar chart
plt.figure(figsize=(6,4))
metrics = ['MAE (ml)', 'R2 Score']
values = [mae, r2]
# To keep bar scale sensible, plot R2 on [0,1] and MAE as absolute; annotate values
plt.bar(metrics, values)
plt.title("Model Performance Metrics")
for i, v in enumerate(values):
    plt.text(i, v + 0.02 * max(values), f"{v:.3f}", ha='center', fontsize=10)
plt.ylabel("Value")
plt.tight_layout()
plt.savefig(os.path.join(OUTDIR, "performance_metrics.png"))
plt.close()

# 6) Plot: RMSE big text
plt.figure(figsize=(6,3))
plt.axis('off')
plt.text(0.5, 0.5, f"RMSE = {rmse:.3f} ml", horizontalalignment='center',
         verticalalignment='center', fontsize=24, weight='bold')
plt.savefig(os.path.join(OUTDIR, "rmse_text.png"), bbox_inches='tight')
plt.close()

# 7) Predicted vs Actual scatter with y=x line
plt.figure(figsize=(6,6))
plt.scatter(y_test, y_pred, alpha=0.7)
minv = min(min(y_test), min(y_pred))
maxv = max(max(y_test), max(y_pred))
plt.plot([minv, maxv], [minv, maxv], 'k--', linewidth=1)  # y = x
plt.xlabel("Actual TV_recommendation (ml)")
plt.ylabel("Predicted TV_recommendation (ml)")
plt.title("Predicted vs Actual")
plt.text(0.05, 0.95, f"R² = {r2:.3f}\nMAE = {mae:.3f} ml\nRMSE = {rmse:.3f} ml",
         transform=plt.gca().transAxes, verticalalignment='top', bbox=dict(boxstyle="round", fc="w", alpha=0.7))
plt.tight_layout()
plt.savefig(os.path.join(OUTDIR, "predicted_vs_actual.png"))
plt.close()

# 8) Residuals plot
residuals = y_test - y_pred
plt.figure(figsize=(6,4))
plt.scatter(y_pred, residuals, alpha=0.7)
plt.hlines(0, xmin=min(y_pred), xmax=max(y_pred), linestyles='dashed', colors='k')
plt.xlabel("Predicted TV_recommendation (ml)")
plt.ylabel("Residuals (Actual - Predicted)")
plt.title("Residuals vs Predicted")
plt.tight_layout()
plt.savefig(os.path.join(OUTDIR, "residuals_plot.png"))
plt.close()

# 9) Learning curve (uses cross-validation)
cv = 5
train_sizes, train_scores, test_scores = learning_curve(
    model, X, y, cv=cv, train_sizes=np.linspace(0.1, 1.0, 5), random_state=RANDOM_STATE
)
train_scores_mean = np.mean(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)

plt.figure(figsize=(7,5))
plt.plot(train_sizes, train_scores_mean, 'o-', label="Training score")
plt.plot(train_sizes, test_scores_mean, 'o-', label=f"Cross-validation score (cv={cv})")
plt.xlabel("Number of training examples")
plt.ylabel("Score (R²)")
plt.title("Learning Curve")
plt.legend(loc="best")
plt.grid(True, linestyle='--', alpha=0.4)
plt.tight_layout()
plt.savefig(os.path.join(OUTDIR, "learning_curve.png"))
plt.close()

# 10) Feature importance (from RandomForest)
try:
    importances = model.feature_importances_
    feat_names = X_train.columns
    fi = pd.Series(importances, index=feat_names).sort_values(ascending=False)[:20]  # top 20
    plt.figure(figsize=(8,6))
    fi.plot(kind='barh')
    plt.gca().invert_yaxis()
    plt.xlabel("Feature importance")
    plt.title("Top Feature Importances (RandomForest)")
    plt.tight_layout()
    plt.savefig(os.path.join(OUTDIR, "feature_importance.png"))
    plt.close()
except Exception as e:
    print("Feature importance plotting failed:", e)

# 11) Optional: SHAP summary plot (if SHAP is installed); saves shap_summary.png
try:
    import shap
    # Use TreeExplainer for tree-based models
    explainer = shap.TreeExplainer(model)
    # Use sample for speed if dataset is large
    sample = X_test.sample(min(100, len(X_test)), random_state=RANDOM_STATE)
    shap_values = explainer.shap_values(sample)
    # summary_plot writes to matplotlib; use show=False then save
    shap.summary_plot(shap_values, sample, show=False)
    plt.tight_layout()
    plt.savefig(os.path.join(OUTDIR, "shap_summary.png"))
    plt.close()
    print("SHAP summary saved.")
except Exception as e:
    print("SHAP not available or SHAP plotting failed. Error:", e)

# 12) Save a small CSV comparing actual vs predicted for inspection
df_compare = pd.DataFrame({"actual": y_test, "predicted": y_pred, "residual": residuals})
df_compare.to_csv(os.path.join(OUTDIR, "actual_vs_predicted.csv"), index=False)

print(f"All plots and CSV saved in '{OUTDIR}/' directory.")


Model saved to tv_predictor_model.pkl
MAE: 10.3278 ml
RMSE: 13.0555 ml
R2: 0.9384


C:\Users\Admin\AppData\Local\Temp\ipykernel_25804\3260225920.py:163: FutureWarning: The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour and silence this warning.
  shap.summary_plot(shap_values, sample, show=False)


SHAP summary saved.
All plots and CSV saved in 'plots/' directory.
